# Imports

In [ ]:
from PIL import Image, ImageEnhance
from skimage.draw import line
import numpy as np
import matplotlib.pyplot as plt
import sys
import cv2
import os
import pickle
from rembg.bg import remove as remove_background
import cairo

# Parameters

In [ ]:
# Parameters 

# Program parameters
CLEAN_WORKSPACE = True # Clean all output files

# I/O path parameters
INPUT_IMAGE_PATH = "img/aldrin.png"
OUTPUT_IMAGE_PATH = "img_out/" + os.path.splitext(INPUT_IMAGE_PATH[4:])[0]

# Input image preprocessing parameters
CONTRAST_FACTOR = 2.0
INVERSE_INPUT_IMAGE = False
MASK_BACKGROUND_COLOR = 100 # 0-255, 0 means background is ignored, the closer to 255 the more important the background is
MASK_OBJECT_STRENGTH = 50 # 0-255, value added to edges of the mask to emphasize them and object itself
MASK_MAX_EDGES_STRENGTH = 255 # 0-255, maximum value of the mask edges
# Keep sum of MASK_OBJECT_STRENGTH and MASK_MAX_EDGES_STRENGTH to be less than 255 if you don't want to overpower the mask

# Penalty calculation
# The algorithm is based on the following formula:
# penalty = (IC * sum(thread_pixels_in_input_image) + MC * sum(thread_pixels_in_mask)) / amount_of_thread_pixels + WC * thread_pixels_weighted_average
IC = 0.0 # input image coefficient
MC = 0.0 # mask image coefficient
WC = 1.0 # weighted average coefficient
DARKNESS = 170 # zobacz czy da sie to latwo zrobic 

# Visual parameters
NAIL_ARRANGEMENT = "ellipse" # Can be set to: "ellipse" / "rectangle"

In [ ]:
#
# Display 
#
def display_image(image, rotate=False):
    if rotate:
        image = cv2.rotate(image, cv2.ROTATE_90_COUNTERCLOCKWISE)
    display(Image.fromarray(image))

def display_mask(image, rotate=False):
    image = image.copy()
    if rotate:
        image = cv2.rotate(image, cv2.ROTATE_90_COUNTERCLOCKWISE)
    display(Image.fromarray(image))

def display_nails(nails, image, rotate=False):
    canvas = np.ones(image.shape, np.uint8) * 255
    for nail in nails:
        canvas[nail[0], nail[1]] = 0
    display_image(canvas, rotate)
    return canvas

def display_threads(threads, image, rotate=False):
    canvas = np.ones(image.shape, np.uint8) * 255
    for thread in threads:
        canvas = cv2.line(canvas, (thread[0][1], thread[0][0]), (thread[1][1], thread[1][0]), 0, 1)
    display_image(canvas, rotate)
    return canvas

#
# Save
#
def save_threads(threads, image, rotate=False):
    canvas = np.ones(image.shape, np.uint8) * 255
    for thread in threads:
        canvas = cv2.line(canvas, (thread[0][1], thread[0][0]), (thread[1][1], thread[1][0]), 0, 1)
    if rotate:
        canvas = cv2.rotate(canvas, cv2.ROTATE_90_COUNTERCLOCKWISE)
    print("Trying to save image as: " + OUTPUT_IMAGE_PATH[:7] + "_png" + OUTPUT_IMAGE_PATH[7:] + "_" + NAIL_ARRANGEMENT+ "_" + str(len(threads)) + "_threaded.png")
    cv2.imwrite(OUTPUT_IMAGE_PATH[:7] + "_png" + OUTPUT_IMAGE_PATH[7:] + "_" + NAIL_ARRANGEMENT + "_" + str(len(threads)) + "_threaded.png", canvas)

def save_threads_svg(threads, size, rotate):
    print("Trying to save image as: " + OUTPUT_IMAGE_PATH[:7] + "_svg" + OUTPUT_IMAGE_PATH[7:] + "_" + NAIL_ARRANGEMENT+ "_" + str(len(threads)) + "_threaded.svg")
    with cairo.SVGSurface(OUTPUT_IMAGE_PATH[:7] + "_svg" + OUTPUT_IMAGE_PATH[7:] + "_" + NAIL_ARRANGEMENT + "_" + str(len(threads)) + "_threaded.svg", size[1], size[0]) as surface:
        context = cairo.Context(surface)
        context.set_source_rgb(255, 255, 255)
        context.rectangle(0, 0, size[1]-1, size[0]-1)
        context.fill()
        context.set_source_rgb(0, 0, 0)
        for thread in threads:
            if rotate:
                thread = np.flip(thread, axis=1)
            context.move_to(thread[0][0], thread[0][1])
            for i in range(1, len(thread)):
                context.line_to(thread[i][0], thread[i][1])
            context.stroke()


def save_threads_path(threads):
    with open(OUTPUT_IMAGE_PATH[:7] + "_txt" + OUTPUT_IMAGE_PATH[7:] + "_" + NAIL_ARRANGEMENT + "_" + str(len(threads.split())-1) + "_threaded_path.txt", "w") as file:
        file.write(threads)

def save_mask(image, rotate=False):
    print("Trying to save mask as: " + OUTPUT_IMAGE_PATH[:7] + "_png" + OUTPUT_IMAGE_PATH[7:] + "_mask.png")
    canvas = image.copy()
    if rotate:
        canvas = cv2.rotate(image, cv2.ROTATE_90_COUNTERCLOCKWISE)
    cv2.imwrite(OUTPUT_IMAGE_PATH[:7] + "_png" + OUTPUT_IMAGE_PATH[7:] + "_mask.png", canvas)

#
# Image
#
def decide_aspect_ratio(image):
    width, height = image.shape[1], image.shape[0]
    img_ar = width / height
    arr = np.asarray([1/1 , 3/4, 4/3, 4/5, 5/4])
    return arr[(np.abs(arr - img_ar)).argmin()]
    
def center_crop_to_aspect_ratio(image, aspect_ratio):
    width, height = image.shape[1], image.shape[0]
    img_ar = width / height
    if img_ar > aspect_ratio:
        return image[:, int((width - height * aspect_ratio) / 2):int((width + height * aspect_ratio) / 2)]
    else:
        return image[int((height - width / aspect_ratio) / 2):int((height + width / aspect_ratio) / 2), :]

def resize_to_aspect_ratio(image, aspect_ratio):
    if aspect_ratio == 1/1:
        return cv2.resize(image, (1561, 1561))
    elif aspect_ratio == 3/4:
        return cv2.resize(image, (1321, 1761))
    elif aspect_ratio == 4/3:
        return cv2.resize(image, (1761, 1321))
    elif aspect_ratio == 4/5:
        return cv2.resize(image, (1441, 1801))
    elif aspect_ratio == 5/4:
        return cv2.resize(image, (1801, 1441))

def get_input(): 
    input_image = cv2.imread(INPUT_IMAGE_PATH, 0)
    if INVERSE_INPUT_IMAGE:
        input_image = cv2.bitwise_not(input_image)

    aspect_ratio = decide_aspect_ratio(input_image)
    input_image = center_crop_to_aspect_ratio(input_image, aspect_ratio)
    input_image = resize_to_aspect_ratio(input_image, aspect_ratio)

    # Background removal
    is_success, buffor_array = cv2.imencode(".png", input_image)
    byte_image = buffor_array.tobytes()
    output_buffor_array = remove_background(byte_image)
    image_no_bg_alpha = cv2.imdecode(np.frombuffer(output_buffor_array, np.uint8), cv2.IMREAD_UNCHANGED)

    # Edge detection
    image_blur = cv2.GaussianBlur(input_image, (3, 3), 0)
    image_sobel_xy = cv2.convertScaleAbs(cv2.Sobel(src=image_blur, ddepth=cv2.CV_64F, dx=1, dy=1, ksize=5))
    image_edges = np.interp(image_sobel_xy, (image_sobel_xy.min(), image_sobel_xy.max()), (0, MASK_MAX_EDGES_STRENGTH)).astype(np.uint8)

    # Mask creation
    mask = image_edges.copy()
    for i in range(input_image.shape[0]):
        for j in range(input_image.shape[1]):
            if image_no_bg_alpha[i, j][3] == 0:
                mask[i, j] = MASK_BACKGROUND_COLOR
            else:
                mask[i, j] = MASK_BACKGROUND_COLOR + np.clip(mask[i, j] + MASK_OBJECT_STRENGTH, 0, 255)
    
    # Contrast enhancement
    mean = np.uint8(cv2.mean(input_image)[0])
    input_image = cv2.addWeighted(input_image, CONTRAST_FACTOR, np.ones_like(input_image) * mean, 1-CONTRAST_FACTOR, 0.0)
    return input_image, mask, aspect_ratio

def get_new_image(dimensions):
    return np.ones(dimensions, np.uint8) * 255

def draw_line(image, line, color):
    return cv2.line(image, (line[0][1], line[0][0]), (line[1][1], line[1][0]), color, 1)

def draw_point(image, point, color):
    image[point[1], point[0]] = color
    return image

#
# Points
#
def get_data_from_dictionaries(aspect_ratio, nail_arrangement):
    ar = ""
    rotate = False
    if aspect_ratio == 1/1:
        ar = "1x1"
    elif aspect_ratio == 3/4: 
        ar = "4x3"
    elif aspect_ratio == 4/3:
        ar = "4x3"
        rotate = True
    elif aspect_ratio == 4/5:
        ar = "5x4"
    elif aspect_ratio == 5/4:
        ar = "5x4"
        rotate = True

    with open("dictionaries/" + nail_arrangement + "_" + ar + ".dat", "rb") as dictionary_file:
        n, t, ntt, ttp = pickle.load(dictionary_file)

    return n, t, ntt, ttp, rotate
    
def is_on_same_edge(point1, point2, dimensions):
    if point1[0] == 0 and point2[0] == 0:
        return True
    if point1[0] == dimensions[0] - 1 and point2[0] == dimensions[0] - 1:
        return True
    if point1[1] == 0 and point2[1] == 0:
        return True
    if point1[1] == dimensions[1] - 1 and point2[1] == dimensions[1] - 1:
        return True
    return False

def get_other_end_of_thread(thread, nail):
    if np.array_equal(thread[0], nail):
        return thread[1]
    return thread[0]

def get_nail_number(nail, n):
    for i in range(len(n)):
        if np.array_equal(nail, n[i]):
            return i
    return -1


#
# Test 
#
# input_image = get_input_image()

# input_image_edges = get_input_image_edges()
# output_image = get_new_image(input_image.shape)



# print("Input image")
# display_image(input_image)

# print("Input image edges")
# display_image(input_image_edges)

# print("Output image")
# display_image(output_image)



In [ ]:
def get_thread_panelty(thread, ttp, input_image, input_image_mask):
    image_t_vals = input_image[tuple(ttp[thread.tobytes()].T)] # values of pixels in thread in input image
    mask_t_vals = input_image_mask[tuple(ttp[thread.tobytes()].T)] # values of pixels in thread in mask image
    # t_count = len(image_t_vals) # amount of pixels in thread (both in input image and mask image)

    # penalty = (IC * np.sum(image_t_vals) + MC * np.sum(mask_t_vals)) / t_count + WC * np.average(image_t_vals, weights=mask_t_vals)
    
    return np.average(image_t_vals, weights=mask_t_vals)

def get_best_fitting_thread(ntt, current_nail, ttp, input_image, input_image_edges): 
    current_threads = ntt[current_nail.tobytes()]
    best_fitting_thread = current_threads[0]
    best_fitting_thread_penalty = 999999999
    for thread in current_threads:
        penalty = get_thread_panelty(thread, ttp, input_image, input_image_edges)
        if penalty < best_fitting_thread_penalty:
            best_fitting_thread = thread
            best_fitting_thread_penalty = penalty
    return best_fitting_thread, best_fitting_thread_penalty

def produce_thread_art():
    if CLEAN_WORKSPACE:
        for filename in os.listdir("./img_out_txt", "./img_out_png", "./img_out_svg"):
            if filename.endswith(".jpg") or filename.endswith(".png") or filename.endswith(".jpeg") or filename.endswith(".svg"):
                os.remove(os.path.join("./img_out_txt", filename))

    print("Producing " + NAIL_ARRANGEMENT + " thread art for: ", INPUT_IMAGE_PATH)
    
    input_image, input_image_mask, aspect_ratio = get_input()
    output_image = get_new_image(input_image.shape)
    thread_path = ""

    print("Parameters:  \tAspect Ratio: " + str(aspect_ratio) + 
                        "\t Mask BG: " + str(MASK_BACKGROUND_COLOR) + 
                        "\t Mask OBJ: " + str(MASK_OBJECT_STRENGTH) + 
                        "\t Mask EDGE: " + str(MASK_MAX_EDGES_STRENGTH))

    n, t, ntt, ttp, rotate = get_data_from_dictionaries(aspect_ratio, NAIL_ARRANGEMENT)

    if rotate: # if dictionary exists for rotated image, use it
        input_image = cv2.rotate(input_image, cv2.ROTATE_90_CLOCKWISE)
        input_image_mask = cv2.rotate(input_image_mask, cv2.ROTATE_90_CLOCKWISE)
        output_image = cv2.rotate(output_image, cv2.ROTATE_90_CLOCKWISE)

    used_threads = []
    current_nail = ntt[list(ntt)[0]][0][0] # start at more suitable one?
    thread_path = thread_path + str(get_nail_number(current_nail, n)) + " "
    current_input_image = input_image.copy()

    print("Input image: ")
    display_image(current_input_image, rotate)
    
    print("Input image mask: ")
    display_mask(input_image_mask, rotate)
    save_mask(input_image_mask, rotate)

    for i in range(1, 4001):
        current_threads = ntt[current_nail.tobytes()]
        if len(current_threads) == 0:
            print("No threads for nail", current_nail)
            break # maybe figure out a way to continue by going back to previous nail and trying different thread

        selected_thread, selected_thread_penalty = get_best_fitting_thread(ntt, current_nail, ttp, current_input_image, input_image_mask)
        next_nail = get_other_end_of_thread(selected_thread, current_nail)
        thread_path = thread_path + str(get_nail_number(next_nail, n)) + " "

        used_threads.append(selected_thread)

        ntt[current_nail.tobytes()] = np.array([a for a, skip in zip(ntt[current_nail.tobytes()], 
                                        [np.allclose(a, selected_thread) for a in ntt[current_nail.tobytes()]]) if not skip])
        ntt[next_nail.tobytes()] = np.array([a for a, skip in zip(ntt[next_nail.tobytes()], 
                                        [np.allclose(a, selected_thread) for a in ntt[next_nail.tobytes()]]) if not skip])
        
        # saves = [1000, 2000, 3000, 3500, 4000, 5000]
        saves = [1000, 1500, 2000, 2500, 3000, 3500]
        if i in saves:
            print("Current nail #", i, ": ", str(current_nail))
            save_threads(used_threads, output_image, rotate)
            save_threads_svg(used_threads, output_image.shape, True)
            save_threads_path(thread_path)
            display_threads(used_threads, output_image, rotate)

        draw_line(current_input_image, selected_thread, 255)
        current_nail = next_nail

    return used_threads
    
    # return display_threads(used_threads, output_image, rotate)

# # Run for all in DIR
# for filename in os.listdir("./img/test/"):
#     if filename.endswith(".jpg") or filename.endswith(".png") or filename.endswith(".jpeg"):
#         INPUT_IMAGE_PATH = "img/test/" + filename
#         OUTPUT_IMAGE_PATH = "img_out/" + os.path.splitext(INPUT_IMAGE_PATH[4:])[0]
#         produce_thread_art()


# Run for one image specified in parameters
# threads_art = produce_thread_art()


MASK_BACKGROUND_COLOR = 50
MASK_OBJECT_STRENGTH = 100 
MASK_MAX_EDGES_STRENGTH = 255
threads_art = produce_thread_art()
